In [1]:
# !conda install -c conda-forge ipympl

# If using JupyterLab
!conda install -c conda-forge nodejs
!jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-matplotlib

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Building jupyterlab assets (build:prod:minimize)


In [1]:
import numpy as np
from utils.data_loading import read_csv
from modules.coincidence_counts import count_coincidences

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]
plt.rcParams.update({'figure.max_open_warning': 0})
%matplotlib widget

# Characterization of the single photon source

## Coincidence counts as a function of coincidence window width

In [3]:
sync, time = read_csv('measurements/coincidence_counts/window_width/5_2_1_window_width.csv')

In [4]:
# time includes the timestamp of each event
time[0:10]

array([1255892, 2579582, 2636980, 3260015, 3903854, 4019483, 4908083,
       4987669, 6296538, 6653431], dtype=uint64)

In [5]:
total_time = time[-1]
print('total measurement time: {} s'.format(total_time*1e-12))

total measurement time: 14.999998040321 s


In [6]:
# sync contains the channel id
sync[0:10]

array([1, 1, 2, 1, 2, 1, 2, 1, 2, 2], dtype=uint8)

In [17]:
coinc_windows = np.arange(0, 1e4, 1e3)
coinc_windows = np.append(coinc_windows, np.arange(1e4, 85000, 1e4))

coinc_rate = np.array([], dtype=np.float64)
accidental_coinc_rate = np.array([], dtype=np.float64)

for coinc_window in coinc_windows:
    coinc_rate = np.append(coinc_rate, count_coincidences(time, sync, coinc_window=coinc_window)/(total_time*1e-3))
    accidental_coinc_rate = np.append(accidental_coinc_rate, coinc_window*1e-3*np.sum(np.where(sync == 1, 1, 0))*np.sum(np.where(sync == 2, 1, 0))/(total_time*1e-3)**2)

In [18]:
fig, ax = plt.subplots()
ax.scatter(coinc_windows/1000, coinc_rate, marker='+')
ax.scatter(coinc_windows/1000, accidental_coinc_rate, marker='x')


ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.set_xlabel('Coincidence window [ns]')
ax.set_ylabel('Coincidence rate [1/ns]') 

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Coincidence counts as a function of pump power

In [26]:
voltages = {0.2:"V0020.csv", 0.39:"V0390.csv", 0.8:"V0800.csv", 1.2:"V120.csv", 1.61:"V161.csv"}

coinc_rate = np.array([], dtype=np.float64)
accidental_coinc_rate = np.array([], dtype=np.float64)

coinc_window = 5000

for file in voltages.values():
    sync, time = read_csv('measurements/coincidence_counts/pump_power/5_2_1_'+file)
    
    total_time = time[-1]
    
    coinc_rate = np.append(coinc_rate, count_coincidences(time, sync, coinc_window=coinc_window)/(total_time*1e-3))
    accidental_coinc_rate = np.append(accidental_coinc_rate, coinc_window*1e-3*np.sum(np.where(sync == 1, 1, 0))*np.sum(np.where(sync == 2, 1, 0))/(total_time*1e-3)**2)    

In [27]:
fig, ax = plt.subplots()
color = 'tab:red'
ax.scatter(voltages.keys(), coinc_rate, marker='+', color=color)
ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.tick_params(axis='y', labelcolor=color)
ax.set_xlabel('Spannung [V]')
ax.set_ylabel('Koinzidenzrate [1/ns]') 

color = 'tab:blue'
ax_2 = ax.twinx()
ax_2.set_ylabel('Zufällige Koinzidenzrate [1/ns]')
ax_2.scatter(voltages.keys(), accidental_coinc_rate, marker='x', color=color)
ax_2.tick_params(axis='y', labelcolor=color)
fig.tight_layout()

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Coincidence counts as a function of pump polarization

In [23]:
import os

data_dir = 'measurements/coincidence_counts/pump_polarization'

polarizations = np.array([])
coinc_rate = np.array([], dtype=np.float64)

coinc_window = 5000

for file_name in os.listdir(data_dir):
    
    base_name, ext = os.path.splitext(file_name)
    if ext != '.csv':
        continue
    
    # we named each file d<degrees>.csv so we can just extract the value
    deg = int(base_name.split('d')[-1])
    
    polarizations = np.append(polarizations, deg)
    
    sync, time = read_csv(os.path.join(data_dir, file_name))
    
    total_time = time[-1]
    
    coinc_rate = np.append(coinc_rate, count_coincidences(time, sync, coinc_window=coinc_window)/(total_time*1e-3))

In [24]:
fig, ax = plt.subplots()
ax.plot(polarizations, coinc_rate, linestyle='--', marker='o')

ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
ax.set_xlabel('Rotation angle')
ax.set_ylabel('Coincidence rate [1/ns]') 

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …